In [27]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat 

In [65]:
class Dataset:
    def __init__(self,
                 file_path,
                 window_start_point = 0.5,
                 window_end_point = 2.5):
        self.data = loadmat(file_name=file_path)
        self.window_start_point = window_start_point
        self.window_end_point = window_end_point
        self.sampling_frequency = self.data['nfo']['fs'][0][0][0][0]
        self.EEGsignal = self.data['cnt'].T
        self.n_channels, self.n_samples = self.EEGsignal.shape
        self.trial_onsets = self.data['mrk'][0][0][0]
        self.trial_codes = self.data['mrk'][0][0][1]
        self.labels = np.zeros(self.n_samples)
        self.labels[self.trial_onsets] = self.trial_codes
        self.class_labels = [x[0] for x in self.data['nfo']['classes'][0][0][0]]
        self.n_classes = len(self.class_labels)
        self.n_trails = len(self.trial_onsets)
        self.window = np.arange(int(self.window_start_point*self.sampling_frequency), int(self.window_end_point*self.sampling_frequency))
        self.n_samples_per_trail = len(self.window)
        self.trials = {}
        for label, code in zip(self.class_labels, np.unique(self.trial_codes)):
            class_onsets = self.trial_onsets[self.trial_codes == code]
            self.trials[label] = np.zeros((len(class_onsets), self.n_channels, self.n_samples_per_trail))
            for i, oneset in enumerate(class_onsets):
                self.trials[label][i, :, :] = self.EEGsignal[:, self.window + oneset]

    
        